In [1]:
rm(list=ls())

In [2]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,610588,32.7,1320457,70.6,1302051,69.6
Vcells,1141182,8.8,8388608,64.0,1821309,13.9


In [3]:
# Uncomment and run to install packages if needed
# install.packages("tidyverse") 
# install.packages("cluster")
# install.packages("forecast")
# install.packages("Metrics")
# install.packages("biglm")
# install.packages("tseries")
# install.packages("scoringutils")

In [4]:
library(tidyverse)
library(parallel)
library(cluster)
library(forecast)
library(Metrics)
library(biglm)
library(tseries)
library(scoringutils)

Warning message:
“‘timedatectl’ indicates the non-existent timezone name ‘n/a’”
Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘Metrics’


The following object is masked from ‘package:forecast’:

    accuracy


Loading required package: DBI

Note: scoringutils 1.0.0 introduces a lot of breaking changes and we apologise for any inconvenience. If you pr

# Read In Data

In [5]:
clust_assign <- read_csv("Results/Unseen Sensor/clust_assign.csv")

New names:
• `` -> `...1`
Rows: 4 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (5): ...1, rand, catch22, tsfeat, dtw

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
fnames <- list.files("Data/Unseen Sensor/Processed/", pattern="*.csv", full.names=TRUE)
total_df_list <- lapply(fnames, read_csv)

Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delim

In [7]:
total_df_list <- lapply(total_df_list, function(x) x %>% arrange(timestamp))    

In [8]:
# Read in the start and end points for each time series from csv
start_end <- read_csv("start_end_points_unseen.csv")
starting_points <- start_end$start
ending_points <- start_end$end

Rows: 4 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
# For each df in our list, create a row number column called 'rn' - this will allow us to find the starting point
# for each sample using the above starting_points array
total_df_list <- lapply(total_df_list, function(x) x %>% mutate(rn = row_number()))

In [10]:
# Sample each df in the list according to its corresponding starting and ending point - this creates 12-week
# long samples of each data frame which will be used for modeling and testing
total_df_list_samples <- lapply(1:length(total_df_list), 
                                function(x) total_df_list[[x]] %>% 
                                    filter(rn >= starting_points[x]) %>%
                                    filter(rn <= ending_points[x]))

In [11]:
# Add a new column to each data frame to denote whether each row should be part of the training set, validation set
# or test set - the first 8 weeks are designated as train, the next 2 as val, and the final 2 as test. It may be 
# the case that train and val get used for training, depending on the modeling task
total_df_list_samples <- lapply(total_df_list_samples, 
                                function(x) x %>% mutate(rn = row_number()) %>%
                                    mutate(train_val_test = ifelse(rn <= (96*7*8), 
                                                                   "train", 
                                                                   ifelse(rn <= (96*7*10), 
                                                                          "val", 
                                                                          "test")))
                                
                               )

In [12]:
set.seed(54321)

# Functions for Global AR

In [13]:
lag_embed <- function(data, lag=1) {
    # This function creates a lag emebedded matrix at the provided lag level. It assumes there is a train_val_test
    # in the input data frame, and returns a list of the training, validaiton, and test data as lag matrices
    
    # Compute the number of rows of test and validation data
    n_test <- nrow(data %>% filter(train_val_test=="test"))
    n_val <- nrow(data %>% filter(train_val_test=="val"))
    
    # Lag embed the entire time series - note we use the input lag+1 to account for the target variable
    embed_mat <- embed(data$total_volume, (lag+1))

    # Get the total numebr of rows in the lag matrix
    n_mat <- nrow(embed_mat)
    
    # Based on the number of rows in the matrix and the number of test and validation samples,
    # compute the indexes which correspond to test and validation
    test_idx <- rev(1:n_mat)[1:n_test]
    val_idx <- rev(1:n_mat)[-c(1:n_test)][1:n_val]
    
    # Remoe the test and val indexes to get the training matrix
    train_mat <- embed_mat[-c(val_idx, test_idx),]
    
    # Filter using the above computed indexes to get the validaiton and test matrices
    val_mat <- embed_mat[val_idx,]
    test_mat <- embed_mat[test_idx,]
    
    # Return a list of the trianing, validation, and test lag-embedded matrices
    return_list <- list("train"=train_mat, "val"=val_mat, "test"=test_mat)
}

In [14]:
global_ar_least_squares_pred <- function(model,
                                         test_list,
                                         return_preds=TRUE,
                                         return_means=TRUE
                                        ) 
{
    # Function which takes in a model and a list of test matrices and makes predictions 
    # and computes performance measurements on that test data
    
    # Rename the model object
    big_lm_mod <- model
    
    # Initialize an empty df for storing the predictions
    test_preds <- data.frame()
    # For each matrix in the test list
    for (test_t in 1:length(test_list)) {
        # Compute the one-step ahead predictions
        test_pred <- data.frame(forec=predict(big_lm_mod, newdata=test_list[[test_t]]), ts_index=test_t)
        # Append those to the test_pred df
        test_preds <- rbind(test_preds, test_pred)
    }

    # Initialize empty lists for rmse, mae, and means
    test_rmse_ls <- c()
    test_mae_ls <- c()
    test_means <- c()
    
    # Loop through each time series in the data set and
    for (idx in 1:length(test_list)) {
        
        # Compute the mean and append to the list of means
        test_mean_i <- mean(test_list[[idx]][,1])
        test_means <- c(test_means, test_mean_i)
        
        # Compute the prediction RMSE and append to the list of RMSE
        rmse_test_i <- rmse(test_list[[idx]][,1], (test_preds %>% filter(ts_index==idx))$forec)
        test_rmse_ls <- c(test_rmse_ls, rmse_test_i)
    
        # Copmute the MAE of the predictions and append to list
        mae_test_i <- mae(test_list[[idx]][,1], (test_preds %>% filter(ts_index==idx))$forec)
        test_mae_ls <- c(test_mae_ls, mae_test_i)
    }
    
    # Set up the return_list to include the rmse and mae lists
    return_list <- list("test_rmse"=test_rmse_ls, "test_mae"=test_mae_ls)
    
    # If the user specifies, return the test set predicitons and the target means as well
    if (return_preds) {
        return_list <- append(return_list, list("test_preds"=test_preds))
    }
    
    if (return_means) {
        return_list <- append(return_list, list("test_means"=test_means))
    }
    
    # Return the return_list
    return_list        
}

In [15]:
bootstrap_pred_int <- function(predictions,
                               residuals,
                               n_boot,
                               actuals=NULL
                              ) 
{
    # Function to compute prediction intervals using bootstrapping with model residuals
    # function takes in the predicted values, the model residuals, the number of bootstrap samples to use,
    # and an optional set of true values (used for computing prediction interval scores)
    
    # Intialize empty lists for the hi and lo components of the 80% nad 95% PIs
    boot_lo_80 <- c()
    boot_hi_80 <- c()
    boot_lo_95 <- c()
    boot_hi_95 <- c()

    # Loop through every row in the predictions
    for (i in 1:nrow(predictions)){
        # Reset the seed to the loop counter - could make this more random, but this is for reproducibility
        set.seed(i)
        
        # Sample with replacement from the residuals n_boot times
        boot_samples <- sample(residuals, n_boot, replace=TRUE)
        # Add the predicted value to each sample
        boot_preds <- boot_samples + predictions$forec[i]
        
        # Compute the PIs as the quantiles of these pred+boot values
        boot_pred_lo_80 <- quantile(boot_preds, 0.1)
        boot_pred_hi_80 <- quantile(boot_preds, 0.9)

        boot_pred_lo_95 <- quantile(boot_preds, 0.025)
        boot_pred_hi_95 <- quantile(boot_preds, 0.975)
        
        # Append the PI values to the lists
        boot_lo_80 <- c(boot_lo_80, boot_pred_lo_80)
        boot_hi_80 <- c(boot_hi_80, boot_pred_hi_80)

        boot_lo_95 <- c(boot_lo_95, boot_pred_lo_95)
        boot_hi_95 <- c(boot_hi_95, boot_pred_hi_95)        
    }
    
    # Create a copy of the input df called predicitons
    pred_w_boot_int_df <- predictions
    
    # Add columns to the df for the PIs
    pred_w_boot_int_df$lo_80 <- boot_lo_80
    pred_w_boot_int_df$hi_80 <- boot_hi_80
    pred_w_boot_int_df$lo_95 <- boot_lo_95
    pred_w_boot_int_df$hi_95 <- boot_hi_95
    
    # If the true values are included
    if (!is.null(actuals)){
        
        # Add a column for the true values
        pred_w_boot_int_df$true_values <- actuals
        
        # Compute the 80% and 95% interval scores and add them as columns in the df
        pred_w_boot_int_df <- pred_w_boot_int_df %>%
            mutate(int_score_80 = interval_score(true_values=true_values,
                                                 lower=lo_80,
                                                 upper=hi_80,
                                                 interval_range=80,
                                                 weigh=FALSE
                                                )) %>% 
            mutate(int_score_95 = interval_score(true_values=true_values,
                                                 lower=lo_95,
                                                 upper=hi_95,
                                                 interval_range=95,
                                                 weigh=FALSE
                                                ))
    }
    
    # Return the final set of preds with PIs (and interval scores if applicable)
    pred_w_boot_int_df
    
}

# Global AR Model for Full Data Set

In [16]:
# Create the final lag embedd matrix list using the lag of 840
embed_mat_list <- lapply(total_df_list_samples,
                     function(x) lag_embed(x, lag=840))

# Since we are training the full model now, we will combine the train and validation data to fit the model and only
# test on the test data - thus, we split these up into two separate lists of lag matrices
embed_train_val_list <- lapply(embed_mat_list,
                           function(x) rbind(data.frame(x$train), data.frame(x$val)))
 
embed_test_list <- lapply(embed_mat_list,
                          function(x) data.frame(x$test))   

In [17]:
full_model <- readRDS("Results/Global/Pooled Regression/Full/global_ar_model.RData")

In [18]:
# Make predicitons on the test data using this model
global_ar_full <- global_ar_least_squares_pred(full_model$model, embed_test_list, TRUE, TRUE)

In [19]:
# Create a data frame of the test data
test_data_full <- do.call("rbind", embed_test_list)

In [20]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_test <- bootstrap_pred_int(global_ar_full$test_preds,
                                full_model$train_residuals,
                                n_samp,
                                test_data_full[,1])     

In [21]:
# Examine the model performance - it is quite similar to the training data, which is good
mean(global_ar_full$test_rmse)

[1] 38.81907

In [22]:
mean(global_ar_full$test_mae)

[1] 25.90133

In [23]:
mean(global_ar_full$test_rmse/global_ar_full$test_means)

[1] 0.1268307

In [24]:
mean(global_ar_full$test_mae/global_ar_full$test_means)

[1] 0.08513731

In [25]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)

[1] 159.3225

In [26]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
global_ar_full$test_means)

[1] 0.5046831

In [27]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)

[1] 297.4774

In [28]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
global_ar_full$test_means)

[1] 0.8992548

# Global AR Model for Highway System

In [29]:
highways_england_model <- readRDS("Results/Global/Pooled Regression/Highway System/global_ar_model_country_1.RData")

In [30]:
# Make predicitons on the test data using this model
global_ar_england_pred <- global_ar_least_squares_pred(highways_england_model$model, embed_test_list, TRUE, TRUE)

In [31]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_test_england <- bootstrap_pred_int(global_ar_england_pred$test_preds,
                                        highways_england_model$train_residuals,
                                        n_samp,
                                        test_data_full[,1])    

In [32]:
# Examine the model performance - it is quite similar to the training data, which is good
mean(global_ar_england_pred$test_rmse)

[1] 38.89827

In [33]:
mean(global_ar_england_pred$test_mae)

[1] 26.02497

In [34]:
mean(global_ar_england_pred$test_rmse/global_ar_england_pred$test_means)

[1] 0.1271928

In [35]:
mean(global_ar_england_pred$test_mae/global_ar_england_pred$test_means)

[1] 0.08545461

In [36]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)

[1] 157.2592

In [37]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
global_ar_england_pred$test_means)

[1] 0.5204781

In [38]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)

[1] 286.931

In [39]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
global_ar_england_pred$test_means)

[1] 0.9499916

# Global AR Random Clustering

In [40]:
clust_assign$rand

[1] 3 2 1 4

In [41]:
rand_clust_mod3 <- readRDS("Results/Global/Pooled Regression/Random Cluster/global_ar_model_cluster_3.RData")
rand_clust_mod2 <- readRDS("Results/Global/Pooled Regression/Random Cluster/global_ar_model_cluster_2.RData")
rand_clust_mod1 <- readRDS("Results/Global/Pooled Regression/Random Cluster/global_ar_model_cluster_1.RData")
rand_clust_mod4 <- readRDS("Results/Global/Pooled Regression/Random Cluster/global_ar_model_cluster_4.RData")

In [42]:
# Make predicitons on the test data using this model
test_pred_clust3 <- global_ar_least_squares_pred(rand_clust_mod3$model, embed_test_list[1], TRUE, TRUE)

test_pred_clust2 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[2], TRUE, TRUE)

test_pred_clust1 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[3], TRUE, TRUE)

test_pred_clust4 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[4], TRUE, TRUE)

In [43]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_testclust3 <- bootstrap_pred_int(test_pred_clust3$test_preds,
                                      rand_clust_mod3$train_residuals,
                                      n_samp,
                                      embed_test_list[[1]][,1]) 

boot_testclust2 <- bootstrap_pred_int(test_pred_clust2$test_preds,
                                      rand_clust_mod2$train_residuals,
                                      n_samp,
                                      embed_test_list[[2]][,1]) 

boot_testclust1 <- bootstrap_pred_int(test_pred_clust1$test_preds,
                                      rand_clust_mod1$train_residuals,
                                      n_samp,
                                      embed_test_list[[3]][,1]) 

boot_testclust4 <- bootstrap_pred_int(test_pred_clust4$test_preds,
                                      rand_clust_mod4$train_residuals,
                                      n_samp,
                                      embed_test_list[[4]][,1]) 

In [44]:
mean(c(test_pred_clust3$test_rmse, test_pred_clust2$test_rmse, 
       test_pred_clust1$test_rmse, test_pred_clust4$test_rmse))

[1] 39.1409

In [45]:
mean(c(test_pred_clust3$test_mae, test_pred_clust2$test_mae,
       test_pred_clust1$test_mae, test_pred_clust4$test_mae))

[1] 26.21532

In [46]:
mean(c(test_pred_clust3$test_rmse, test_pred_clust2$test_rmse, test_pred_clust1$test_rmse, test_pred_clust4$test_rmse)/
     c(test_pred_clust3$test_means, test_pred_clust2$test_means, test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.1282712

In [47]:
mean(c(test_pred_clust3$test_mae, test_pred_clust2$test_mae, test_pred_clust1$test_mae, test_pred_clust4$test_mae)/
     c(test_pred_clust3$test_means, test_pred_clust2$test_means, test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.08664577

In [48]:
mean((rbind(boot_testclust3, boot_testclust2, boot_testclust1, boot_testclust4))$int_score_80)

[1] 158.8624

In [49]:
mean(c(mean(boot_testclust3$int_score_80), mean(boot_testclust2$int_score_80), 
mean(boot_testclust1$int_score_80), mean(boot_testclust4$int_score_80))/
c(test_pred_clust3$test_means, test_pred_clust2$test_means, 
  test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.5098389

In [50]:
mean((rbind(boot_testclust3, boot_testclust2, boot_testclust1, boot_testclust4))$int_score_95)

[1] 292.6744

In [51]:
mean(c(mean(boot_testclust3$int_score_95), mean(boot_testclust2$int_score_95), 
mean(boot_testclust1$int_score_95), mean(boot_testclust4$int_score_95))/
c(test_pred_clust3$test_means, test_pred_clust2$test_means, 
  test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.9059299

# Global AR Catch22

In [52]:
clust_assign$catch22

[1] 1 1 4 4

In [53]:
catch22_mod1 <- readRDS("Results/Global/Pooled Regression/Catch22 KMeans/global_ar_model_kmeans_catch22_clust_1.RData")

catch22_mod4 <- readRDS("Results/Global/Pooled Regression/Catch22 KMeans/global_ar_model_kmeans_catch22_clust_4.RData")

In [54]:
# Make predicitons on the test data using this model
test_pred_catch22_clust1 <- global_ar_least_squares_pred(catch22_mod1$model, 
                                                         embed_test_list[1:2], TRUE, TRUE)

test_pred_catch22_clust4 <- global_ar_least_squares_pred(catch22_mod4$model, 
                                                         embed_test_list[3:4], TRUE, TRUE)

In [55]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_catch22_testclust1 <- bootstrap_pred_int(test_pred_catch22_clust1$test_preds,
                                              catch22_mod1$train_residuals,
                                              n_samp,
                                              do.call(rbind, embed_test_list[1:2])[,1])

boot_catch22_testclust4 <- bootstrap_pred_int(test_pred_catch22_clust4$test_preds,
                                              catch22_mod4$train_residuals,
                                              n_samp,
                                              do.call(rbind, embed_test_list[3:4])[,1])

In [56]:
mean(c(test_pred_catch22_clust1$test_rmse, test_pred_catch22_clust4$test_rmse))

[1] 38.7931

In [57]:
mean(c(test_pred_catch22_clust1$test_mae,test_pred_catch22_clust4$test_mae))

[1] 25.88335

In [58]:
mean(c(test_pred_catch22_clust1$test_rmse, test_pred_catch22_clust4$test_rmse)/
     c(test_pred_catch22_clust1$test_means, test_pred_catch22_clust4$test_means)
     )

[1] 0.1260936

In [59]:
mean(c(test_pred_catch22_clust1$test_mae, test_pred_catch22_clust4$test_mae)/
     c(test_pred_catch22_clust1$test_means, test_pred_catch22_clust4$test_means)
     )

[1] 0.08458972

In [60]:
mean(c(boot_catch22_testclust1$int_score_80, boot_catch22_testclust4$int_score_80))

[1] 149.9397

In [61]:
mean(c((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80,
(boot_catch22_testclust4 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)/
c(test_pred_catch22_clust1$test_means, test_pred_catch22_clust4$test_means)
     )

[1] 0.4789031

In [62]:
mean(c(boot_catch22_testclust1$int_score_95, boot_catch22_testclust4$int_score_95))

[1] 266.3492

In [63]:
mean(c((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95,
(boot_catch22_testclust4 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)/
c(test_pred_catch22_clust1$test_means, test_pred_catch22_clust4$test_means)
     )

[1] 0.8067972

# GlobalAR TSFeat

In [64]:
clust_assign$tsfeat

[1] 1 1 1 1

In [65]:
tsfeat_mod1 <- readRDS("Results/Global/Pooled Regression/TSFeat KMeans/global_ar_model_kmeans_tsfeat_clust_1.RData")

In [66]:
# Make predicitons on the test data using this model
test_pred_tsfeat_clust1 <- global_ar_least_squares_pred(tsfeat_mod1$model, 
                                                        embed_test_list, TRUE, TRUE)

In [67]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_tsfeat_testclust1 <- bootstrap_pred_int(test_pred_tsfeat_clust1$test_preds,
                                             tsfeat_mod1$train_residuals,
                                             n_samp,
                                             test_data_full[,1])

In [68]:
mean(test_pred_tsfeat_clust1$test_rmse)

[1] 38.77228

In [69]:
mean(test_pred_tsfeat_clust1$test_mae)

[1] 25.89013

In [70]:
mean(test_pred_tsfeat_clust1$test_rmse/
    test_pred_tsfeat_clust1$test_means)

[1] 0.1271305

In [71]:
mean(test_pred_tsfeat_clust1$test_mae/
    test_pred_tsfeat_clust1$test_means)

[1] 0.08542864

In [72]:
mean(boot_tsfeat_testclust1$int_score_80)

[1] 156.6058

In [73]:
mean((boot_tsfeat_testclust1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
test_pred_tsfeat_clust1$test_means)

[1] 0.5127753

In [74]:
mean(boot_tsfeat_testclust1$int_score_95)

[1] 289.2041

In [75]:
mean((boot_tsfeat_testclust1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
test_pred_tsfeat_clust1$test_means)

[1] 0.9186435

# GlobalAR DTW

In [76]:
clust_assign$dtw

[1] 2 2 1 1

In [77]:
dtw_clust_mod2 <- readRDS("Results/Global/Pooled Regression/DTW/global_ar_dtw_clust_2.RData")
dtw_clust_mod1 <- readRDS("Results/Global/Pooled Regression/DTW/global_ar_dtw_clust_1.RData")

In [78]:
# Make predicitons on the test data using this model
# Make predicitons on the test data using this model
test_pred_dtw_clust2 <- global_ar_least_squares_pred(dtw_clust_mod2$model,
                                                     embed_test_list[1:2], TRUE, TRUE)

test_pred_dtw_clust1 <- global_ar_least_squares_pred(dtw_clust_mod1$model,
                                                     embed_test_list[3:4], TRUE, TRUE)

In [79]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000

boot_dtwclust_2 <- bootstrap_pred_int(test_pred_dtw_clust2$test_preds,
                                      dtw_clust_mod2$train_residuals,
                                      n_samp,
                                      do.call(rbind, embed_test_list[1:2])[,1]) 


boot_dtwclust_1 <- bootstrap_pred_int(test_pred_dtw_clust1$test_preds,
                                      dtw_clust_mod1$train_residuals,
                                      n_samp,
                                      do.call(rbind, embed_test_list[3:4])[,1]) 

In [80]:
mean(c(test_pred_dtw_clust1$test_rmse, test_pred_dtw_clust2$test_rmse))

[1] 38.70527

In [81]:
mean(c(test_pred_dtw_clust1$test_mae, test_pred_dtw_clust2$test_mae))

[1] 25.84112

In [82]:
mean(c(test_pred_dtw_clust1$test_rmse, test_pred_dtw_clust2$test_rmse)/
     c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means)
    )

[1] 0.1260073

In [83]:
mean(c(test_pred_dtw_clust1$test_mae, test_pred_dtw_clust2$test_mae)/
     c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means)
    )

[1] 0.08460906

In [84]:
mean((rbind(boot_dtwclust_1, boot_dtwclust_2))$int_score_80)

[1] 146.2847

In [85]:
mean(c((boot_dtwclust_1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80,
(boot_dtwclust_2 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)/
c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means))

[1] 0.4724448

In [86]:
mean((rbind(boot_dtwclust_1, boot_dtwclust_2))$int_score_95)

[1] 250.4527

In [87]:
mean(c((boot_dtwclust_1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95,
(boot_dtwclust_2 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)/
c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means))

[1] 0.7747156